In [3]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import copy
import re
from time import sleep
from selenium import webdriver

In [337]:
carmax_url = 'https://www.carmax.com/cars/ford'
#carmax_url = 'https://www.carmax.com/'
#carmax_url = 'https://www.carvana.com/cars/ford'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [443]:
client = MongoClient('localhost', 27017)
db = client['Capstone1']
pages = db.url

In [440]:
#soup = BeautifulSoup(r.text,'html.parser')

In [441]:
#print(soup.prettify())

In [442]:
Ford_files=['Ford_2013_2016.html','Ford_2017_2020.html']

In [430]:
#soup = BeautifulSoup(open("/home/ali/Downloads/Used Ford 2013-2021 for Sale.html"), "html.parser")
#soup = BeautifulSoup(open("/home/ali/Documents/CapStone_Projects/CapStone1/Ford_2013_2016.html"), "html.parser")
#d=[]
#for idx,website in enumerate(soup.find_all('article','car-tile')):
#    d.append(website.a['href'])
url_links = []
for file in Ford_files:
    soup = BeautifulSoup(open("/home/ali/Documents/CapStone_Projects/CapStone1/" + file), "html.parser")
    for idx,link in enumerate(soup.find_all('article','car-tile')):
        url_links.append(link.a['href'])
        

In [446]:
#Saving into mongo database
#for link in set(url_links):
#    pages.insert_one({'link':link})

# Tests

In [57]:
#new_url = 'https://www.carmax.com/car/19651702'
#new_url = 'https://www.carmax.com/car/19687205'
new_url = 'https://www.carmax.com/car/19140440'
#new_url = 'https://www.carmax.com/car/19935122'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [58]:
r_1 = requests.get(new_url,headers=headers)

In [59]:
soup = BeautifulSoup(r_1.text,'html.parser')
#print(soup.prettify())

### Car Mileage

In [60]:
car_mileage = soup.find_all('div','value')[1].text.strip()
car_mileage

'62K miles'

### Car Price

In [61]:
car_price = soup.find_all('div','value')[0].text.strip()
car_price

'$33,998*'

### Car MPG

In [67]:
#soup.find_all('div','kmx-typography--body-1')[1].text
#car_mpg = soup.find('div','mpg').text
#car_mpg
#soup.find('div','mpg').text
car_mpg =[None if soup.find_all('div','mpg')==[] else soup.find('div','mpg').text]
#soup.find_all('div','mpg')
car_mpg

['16/22 mpg']

### Model year

In [13]:
car_year = soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[0]

### Model

In [14]:
car_model = soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[1]
#car_model

### Make

In [15]:
Car_make = soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__model-trim').text.strip()

### Color

In [16]:
#soup.find_all('div','specifications-overview')
#soup.find_all('div','specifications-overview')
#specifications-overview > div:nth-child(2) > div:nth-child(5) > p:nth-child(2)
#soup.select_one('#specifications-overview > div:nth-child(2) > div:nth-child(5) > p:nth-child(2)').text
a = soup.find_all('script')[6]
a = str(a)
car_color = a.split('ExteriorColor":"')[1].split('"')[0]

### Transmission

In [17]:
car_transmission = a.split('Transmission":"')[1].split('"')[0]
car_transmission

'Automatic'

### Engine Size

In [18]:
car_engineSize = float(a.split('EngineSize":"')[1].split('L')[0])
car_engineSize

5.0

### Horsepower

In [19]:
car_HorsePower = float(a.split('Horsepower":')[1].split(',')[0])
car_HorsePower

385.0

### EngineCylinder

In [20]:
car_EngineCylinder = int(a.split('EngineCylinders":')[1].split(',')[0])
car_EngineCylinder

8

### MSRP price

In [21]:
car_msrp = float(a.split('Msrp":')[1].split(',')[0])
car_msrp

34300.0

In [22]:
a

'<script>var viewModel={vehicle:{"StockNumber":19935122,"Vin":"1FTEW1C51JKD38174","Year":2018,"Make":"Ford","Model":"F150","Body":"4D Crew Cab","Trim":"Lariat","Color":{"ExteriorColor":"Black","InteriorColor":"Black","NormalizedExteriorColor":"Black","NormalizedInteriorColor":"Black"},"AccidentCount":0,"OwnerCount":2,"Price":{"BasePrice":36998.0,"Msrp":34300.0,"RebateAmount":null,"RebateEndDate":null},"Certifications":[],"Mileage":40067,"IsNew":false,"IsNewArrival":false,"IsSaleable":false,"IsSold":false,"ExternalIds":{"IntelVehicleNumber":null,"MakeCode":"FO","ModelCode":"F150","ManufacturersCode":null},"ManufacturersWarranty":null,"Specifications":{"EngineSize":"5.0L","EngineType":"Turbo Gas","EngineTorque":385,"EngineTorqueRpm":3850,"Horsepower":385,"HorsepowerRpm":5750,"EngineCylinders":8,"FuelType":"Gas","MpgCity":null,"MpgHighway":null,"DriveTrain":"2WD","Transmission":"Automatic"},"Reconditioning":{"NewTireCount":2,"ReconditioningItems":["Replaced Two Tires"]},"StoreId":7114,"Lo